# Evaluate IR (Information Retrieval)

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService

provider = ProviderService()

In [2]:
# QA_REPO = "BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong"
# QA_SPLIT = "INDEX.medium_index_TDT"
# qa_dataset = load_dataset(QA_REPO, split=QA_SPLIT)
# qa_dataset

QA_REPO = "BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh"
QA_SPLIT = "train"
qa_dataset = load_dataset(QA_REPO, split=QA_SPLIT)
qa_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id'],
    num_rows: 54
})

## Run IR

In [3]:
qa_dataset[0]['metadata']

'{"shards": 1}'

In [4]:
DATA_REPO = "BroDeadlines/TEST.edu_tdt_proposition_data"
DATA_SPLIT = "INDEX.medium_index_TDT"
index_dataset = load_dataset(DATA_REPO, split="INDEX.medium_index_TDT")

Generating propositon_medium_edu_tdt split:   0%|          | 0/344 [00:00<?, ? examples/s]

Generating INDEX.medium_index_TDT split:   0%|          | 0/344 [00:00<?, ? examples/s]

In [5]:
index_dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

## Update shards

In [7]:
ids = {}

def build_ids(row):
    ids[row['doc_id']] = 1
    return
    
eval_dataset.map(build_ids)
len(ids)

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

144

In [8]:
eval_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation', 'metadata'],
    num_rows: 144
})

In [12]:
shards = {}

def build_shards(row):
    if row['doc_id'] not in ids:
        return
    # shards[row['doc_id']] = row['shards']
    shards[row['doc_id']] = len(row['proposition_list'])
    return
    
index_dataset.map(build_shards)
len(shards)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

144

In [13]:
import json

eval_dataset = eval_dataset.map(lambda e: {**e, "metadata": json.dumps({'shards': shards[e['doc_id']]})})

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [15]:
EVAL_REPO = "BroDeadlines/EVAL.IR_evaluation"
split = "INDEX.medium_index_TDT.proposition.sentence.hybrid"
# split = "INDEX.medium_index_TDT.proposition.hybrid"
eval_dataset.push_to_hub(EVAL_REPO, split=split)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.IR_evaluation/commit/78086836d46c649e532b5ea084e80f4b518d12a6', commit_message='Upload dataset', commit_description='', oid='78086836d46c649e532b5ea084e80f4b518d12a6', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
metas = {}

def build_meta(row):
    metas[row['doc_id']] = row['metadata']
    pass

qa_dataset_updated.map(build_meta)
len(metas)

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

144

In [32]:

qa_dataset_updated.push_to_hub(QA_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/785 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.FQA_tu_van_hoc_duong/commit/293b6e18f98f99f46239da08961939357d5f88c1', commit_message='Upload dataset', commit_description='', oid='293b6e18f98f99f46239da08961939357d5f88c1', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
qa_dataset['train'] = qa_dataset['train'].map(lambda e: {**e, "metadata": ""})

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

In [24]:
qa_dataset['TEST.basic_test_tdt_dataset'] = qa_dataset['TEST.basic_test_tdt_dataset'].remove_columns(['shards'])
# dir(a)

In [32]:
qa_dataset.push_to_hub("BroDeadlines/QA.FQA_tu_van_hoc_duong")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/QA.FQA_tu_van_hoc_duong/commit/c2e94ef7977fb20e5b69ae47e42a4ac2f811a40c', commit_message='Upload dataset', commit_description='', oid='c2e94ef7977fb20e5b69ae47e42a4ac2f811a40c', pr_url=None, pr_revision=None, pr_num=None)

## Run RAG

In [4]:
from src.rag.hg_parent_retriever import HugFaceParentRAG
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.utils.type_utils import get_default_config

config = get_default_config()

# config['vec_index'] = "vec-sentence-propositon_medium_edu_tdt"
# config['txt_index'] = "text-sentence-propositon_medium_edu_tdt"

# config['vec_index'] = "vec-sentence-mini-hub-sentence"
# config['txt_index'] = "text-sentence-mines_sizei-hub-sentence"

# config['vec_index'] = "vec-sentence-compact-tdt-sentence"
# config['txt_index'] = "text-sentence-compact-tdt-sentence"
# config['vec_index'] = "vec-raptor-ueh-data-tree"
# config['txt_index'] = "text-raptor-ueh-data-tree"

# config['vec_index'] = "vec-sentence-ueh-sentence"
# config['txt_index'] = "text-sentence-ueh-sentence"

config['vec_index'] = "vec-sentence-ueh-unique"
config['txt_index'] = "text-sentence-ueh-unique"

config['total_k'] = 8
config['llm'] = "gemini-1.0-pro"
config

rag = HugFaceParentRAG(provider=provider, config=config, text_corpora=None)

# rag = HydeHybridSearchRAG(provider=provider, config=config)

In [4]:
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.utils.config_utils import get_gemini_hyde_config

# rag = HydeRAG(provider=provider, index="test-basic_test_tdt_dataset")
# text_idx = "text-raptor-medium_index_tdt_vi"
# vec_idx = "vec-raptor-medium_index_tdt_vi"
vec_idx = "vec-raptor-proposition-medium_index_tdt_vi"
text_idx = "text-raptor-proposition-medium_index_tdt_vi"

text_idx = "text-sentence-compact-tdt-sentence"
vec_idx = "vec-sentence-compact-tdt-sentence"


config = get_gemini_hyde_config()
config.vector_index = vec_idx
config.text_index = text_idx

# print(config.)

# config.vector_index = "vec-raptor-basic_index_tdt_clean"
# config.text_index = "text-raptor-basic_index_tdt_clean"

# config.vector_index = "vec-sentence-propositon_medium_edu_tdt"
# config.text_index = 'text-sentence-propositon_medium_edu_tdt'

# rag = HydeRAG(provider=provider, index=config.vector_index)
rag = HydeHybridSearchRAG(provider=provider,config=config, k=4)

### simple RAG

In [5]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
retriever = rag.ensemble_retriever
model = provider.get_simple_gemini_pro()
# Prompt
prompt = hub.pull("rlm/rag-prompt")
RETRIEVE_DOCS = []

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def store_docs(docs):
    RETRIEVE_DOCS.append(docs)
    return docs

# Chain
rag_chain = (
    {"context": retriever | store_docs | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [5]:
rag.ensemble_retriever

EnsembleRetriever(retrievers=[MyElasticSearchBM25Retriever(client=<Elasticsearch(['https://158.178.243.160:9201', 'https://158.178.243.160:9202', 'https://158.178.243.160:9203'])>, index_name='text-sentence-ueh-unique'), VectorStoreRetriever(tags=['ElasticsearchStore', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.elasticsearch.ElasticsearchStore object at 0x7f95bfb0ce50>, search_kwargs={'k': 4.0, 'fetch_k': 10})], weights=[0.5, 0.5])

In [6]:
def eval(row):
    docs = rag.search(question=row['question'])
    if (docs == None):
        row['evaluation'] = []
        return row
    result = [{'content': d[0].page_content, "doc_id": d[0].metadata['doc_id'], "score": d[1]} for d in docs]
    row['evaluation'] = result
    return row

def eval_no_score(row):
    docs = rag.ensemble_retriever.invoke(row['question'])
    if (docs == None):
        row['evaluation'] = []
        return row
    result = [{'content': d.page_content, "doc_id": d.metadata['doc_id'], "score": 0} for d in docs]
    row['evaluation'] = result
    return row

def eval_no_score_answer(row):
    # answer = rag.invoke(row['question'])
    answer = rag.answer(row['question'])
    docs = rag.retrieve_docs.pop()
    result = [{'content': d.page_content, "doc_id": d.metadata['doc_id'], "score": -1.0} for d in docs]
    row['evaluation'] = result
    row['answer'] = answer
    return row

test_ds = qa_dataset.map(eval_no_score)

Parameter 'function'=<function eval_no_score at 0x7f95bc174ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [7]:
test_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation'],
    num_rows: 54
})

In [8]:
test_ds.filter(lambda row: len(row['evaluation']) > 1 )

Filter:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'evaluation'],
    num_rows: 54
})

In [12]:
import json
def update_meta(row):
    row['metadata'] = json.dumps({"shards": 1})
    return row
    
test_ds_updated = test_ds.map(update_meta)

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [8]:
test_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata', 'evaluation'],
    num_rows: 144
})

In [15]:
def update_score(row):
    tmp = row['evaluation']
    n_tmp = [{**i, "score": -1.1} for i in tmp]
    row['evaluation'] = n_tmp
    return row

test_ds_updated_one = test_ds.map(update_score)

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [10]:
SPLIT

'INDEX.medium_index_TDT'

In [15]:
from src.utils.eval_utils import evaluate_IR, evaluate_IR_RAPTOR

res = evaluate_IR(eval_dataset=test_ds_updated, limit_k=8, shard_key="shards")

In [16]:
res

{'relevant': 1.037037037037037,
 'precision': 0.56,
 'recall': 1.037037037037037,
 'map_score': 0.24446963970773494,
 'relevant_retrieved': 56,
 'num_retrieved': 100}

### RAPTOR: Update Shards

In [4]:
CLUSTER_REPO = "BroDeadlines/TEST.NEW.PART_CLUSTER.raptor.edu_tdt_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.NEW.PART_SUMMERIZE.raptor.edu_tdt_data"
SPLIT = "TEST.medium_tdt_proposition_raptor_vi"

cluster_dataset =  load_dataset(CLUSTER_REPO, split=SPLIT)
summerize_dataset =  load_dataset(SUMMERIZE_REPO, split=SPLIT)

Generating TEST.medium_tdt_raptor split:   0%|          | 0/1925 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_raptor_vi split:   0%|          | 0/1523 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_proposition_raptor_vi split:   0%|          | 0/1570 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_raptor split:   0%|          | 0/332 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_raptor_vi split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_proposition_raptor_vi split:   0%|          | 0/278 [00:00<?, ? examples/s]

In [5]:
easy_shards = {}
hard_shards = {}

def get_shards_cluster_1(row):
    d_id = row['doc_ids']
    if d_id not in hard_shards:
        hard_shards[d_id] = row['hard_shards']
        easy_shards[d_id] = row['easy_shards']
    return row

cluster_dataset.filter(lambda row: row['level_id'] == "tree_1").map(get_shards_cluster_1)

len(easy_shards)

Filter:   0%|          | 0/1570 [00:00<?, ? examples/s]

Map:   0%|          | 0/1302 [00:00<?, ? examples/s]

344

In [20]:
summerize_dataset.map(get_shards_cluster_1)

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', 'easy_shards', 'hard_shards'],
    num_rows: 277
})

In [21]:
sum(easy_shards.values())

1531

In [9]:
def update_shards(row):
    id = row['doc_id']
    return {**row, 'easy_shards': easy_shards[id], 'hard_shards': hard_shards[id]}

test_ds_update = test_ds.map(update_shards)
test_ds_update

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata', 'evaluation', 'easy_shards', 'hard_shards'],
    num_rows: 144
})

In [10]:
REPO = "BroDeadlines/EVAL.NEW.raptor.IR_evaluation"
SPLIT = "TEST.Hyde.medium_tdt_proposition_raptor_vi"

test_ds_update.push_to_hub(REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.NEW.raptor.IR_evaluation/commit/5db35cda4bc3463f3ed5c64c51a336a828b689de', commit_message='Upload dataset', commit_description='', oid='5db35cda4bc3463f3ed5c64c51a336a828b689de', pr_url=None, pr_revision=None, pr_num=None)

### Upload

In [9]:
SPLIT

'INDEX.medium_index_TDT'

In [10]:
test_ds.push_to_hub("BroDeadlines/EVAL.NEW.raptor.IR_evaluation", split="TEST.medium_tdt_raptor_vi")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/725 [00:00<?, ?B/s]

ValueError: Features of the new split don't match the features of the existing splits on the hub: {'question': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'group': Value(dtype='string', id=None), 'doc_id': Value(dtype='string', id=None), 'metadata': Value(dtype='string', id=None), 'evaluation': [{'content': Value(dtype='string', id=None), 'doc_id': Value(dtype='string', id=None), 'score': Value(dtype='float64', id=None)}]} != {'question': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'group': Value(dtype='string', id=None), 'doc_id': Value(dtype='string', id=None), 'metadata': Value(dtype='string', id=None), 'evaluation': [{'content': Value(dtype='string', id=None), 'doc_id': Value(dtype='string', id=None), 'score': Value(dtype='float64', id=None)}], 'easy_shards': Value(dtype='int64', id=None), 'hard_shards': Value(dtype='int64', id=None)}

In [17]:
test_ds

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata', 'evaluation'],
    num_rows: 144
})

In [19]:
test_ds[6]['metadata']

''

# Evaluation IR

In [5]:
EVAL_REPO = "BroDeadlines/EVAL.IR_evaluation"
EVAL_REPO = "BroDeadlines/EVAL.NEW.raptor.IR_evaluation"
# split = "INDEX.medium_index_TDT.proposition.sentence.hybrid"
split = "INDEX.medium_index_TDT.proposition.hybrid"
split = "INDEX.medium_index_TDT.fulltext.clean.8.proposition.sentence.hybrid"
# eval_dataset = load_dataset(EVAL_REPO, split=split)
split = "TEST.Hyde.medium_tdt_proposition_raptor_vi"
eval_dataset = load_dataset(EVAL_REPO, split=split)

Generating TEST.medium_tdt_raptor split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.Hyde.medium_tdt_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.Hyde.medium_tdt_proposition_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

In [6]:
print(SPLIT)
len(eval_dataset)

INDEX.medium_index_TDT


144

In [16]:
KEY = 'INDEX.medium_index_TDT.fulltext.clean.proposition.sentence.hybrid'
KEY = 'INDEX.medium_index_TDT.fulltext.clean.8.proposition.sentence.hybrid'
KEY = 'INDEX.medium_index_TDT.fulltext.clean.2.proposition.sentence.hybrid'
eval = eval_dataset[KEY]

In [17]:
a = eval[0]['evaluation']
len(a)

4

In [7]:
import json

shards = []

def count_shards(row):
    meta = json.loads(row['metadata'])
    shards.append(meta['shards'])
    return row

eval_dataset.map(count_shards)
sum(shards)

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

144

In [18]:
ads = eval_dataset.map(lambda e: {**e, "easy_shards": 1})

In [19]:
ads

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata', 'evaluation', 'easy_shards'],
    num_rows: 144
})

In [21]:
from src.utils.eval_utils import evaluate_IR, evaluate_IR_RAPTOR

# res = evaluate_IR(eval_dataset=eval, limit_k=4)
eval_dataset = ads
res = evaluate_IR_RAPTOR(eval_dataset, 7, 'easy_shards')
res

{'precision': 0.637,
 'recall': 0.854,
 'map_score': 0.273,
 'relevant_retrieved': 123,
 'num_retrieved': 193,
 'nulls': 0}

## RAPTOR

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService

provider = ProviderService()

In [2]:
QA_REPO = "BroDeadlines/EVAL.NEW.raptor.IR_evaluation"
SPLIT = "TEST.medium_tdt_raptor_vi"
test_ds_update = load_dataset(QA_REPO, split=SPLIT)
test_ds_update

Generating TEST.medium_tdt_raptor split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.medium_tdt_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.Hyde.medium_tdt_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating TEST.Hyde.medium_tdt_proposition_raptor_vi split:   0%|          | 0/144 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata', 'evaluation', 'easy_shards', 'hard_shards'],
    num_rows: 144
})

In [3]:
import json 
from src.utils.eval_utils import evaluate_IR_RAPTOR

t_from = 6
t_to = 8
type = "hard"
data = {}
for i in range(t_from, t_to+1):
    res = evaluate_IR_RAPTOR(test_ds_update, i, f'{type}_shards')
    k = f"k_{i}_{type}"
    data[k] = res
json.dumps(data)

'{"k_6_hard": {"precision": 0.58, "recall": 0.19, "map_score": 0.282, "relevant_retrieved": 116, "num_retrieved": 200, "nulls": 0}, "k_7_hard": {"precision": 0.581, "recall": 0.207, "map_score": 0.275, "relevant_retrieved": 126, "num_retrieved": 217, "nulls": 0}, "k_8_hard": {"precision": 0.595, "recall": 0.221, "map_score": 0.271, "relevant_retrieved": 135, "num_retrieved": 227, "nulls": 0}}'